In [1]:
import requests
import pandas as pd
import json
from google_api import api_key
import gmaps
from geopy import distance
import matplotlib.pyplot as plt
import numpy as np

geo_df = pd.read_csv("CoordinateData.csv")
geo_df = geo_df.rename(columns = {"Corrdinates":"Coordinates",
                                  "Average Number Unemployed":"Number Unemployed",
                                  "Average Percent Unemployed":"Percent Unemployed"})
geo_df.head(2)

,Year,State,County,Number Unemployed,Percent Unemployed,Number of Crimes,Total Labor Force,Coordinates
0,2010,AL,Lee,6005.750000,9.025,7107,66764.16667,"(32.5888702, -85.39627689999999)"
1,2011,AL,Lee,5503.583333,8.050,9148,68315.50000,"(32.5888702, -85.39627689999999)"


In [2]:
geo_df["Crimes per Capita"] = geo_df["Number of Crimes"] / geo_df["Total Labor Force"]

In [16]:
geo_df = geo_df.round(5)
len(geo_df["State"])*.1

1671.9

In [6]:
geo_df = geo_df[geo_df["Number of Crimes"] != 0]
geo_df["Number of Crimes"].value_counts()

2        86
4        66
14       62
10       61
20       57
6        56
18       50
12       50
42       49
54       47
16       46
30       45
24       45
44       43
34       42
28       41
52       41
26       41
46       41
22       40
36       38
40       36
50       36
56       36
8        35
58       35
70       35
32       35
78       35
38       35
         ..
9293      1
5211      1
1129      1
5227      1
7290      1
23682     1
5259      1
9357      1
9373      1
5291      1
9389      1
11308     1
1065      1
3112      1
21475     1
953       1
7098      1
3016      1
7114      1
19408     1
46047     1
1001      1
21539     1
5099      1
3064      1
3080      1
80910     1
7194      1
5147      1
2057      1
Name: Number of Crimes, Length: 6412, dtype: int64

In [11]:
y = 0
s = 1
c = 2
nu = 3
pu = 4
nc = 5
tl = 6
coord = 7
cpc = 8

def area_data(geo_year,dist_away):
    number_counties = len(geo_year["County"])
    number_unemployed_column = []
    unemployed_percent_column = []
    number_crimes_column = []
    crime_percent_column = []
    area_counties_column = []
    count_column = []
    highest_unemployment_in_area_column = []
    lowest_unemployment_in_area_column = []
    highest_crime_rate_in_area_column = []
    lowest_crime_rate_in_area_column = []
    for row1 in range(number_counties):
        coord1 = geo_year.iloc[row1,coord]
        number_unemployed1 = geo_year.iloc[row1,nu]
        percent_unemployed1 = geo_year.iloc[row1,pu]
        number_crimes1 = geo_year.iloc[row1,nc]
        total_labor1 = geo_year.iloc[row1,tl]
        crimes_per_capita1 = geo_year.iloc[row1,cpc]
        number_unemployed = []
        unemployed_percent = []
        number_crimes = []
        crime_percent = []
        area_counties = []
        for row2 in range(number_counties):
            coord2 = geo_year.iloc[row2,coord]
            number_unemployed2 = geo_year.iloc[row2,nu]
            percent_unemployed2 = geo_year.iloc[row2,pu]
            number_crimes2 = geo_year.iloc[row2,nc]
            total_labor2 = geo_year.iloc[row2,tl]
            crimes_per_capita2 = geo_year.iloc[row2,cpc]
            if distance.distance(coord1,coord2).miles <= dist_away:
                number_unemployed.append(number_unemployed2)
                unemployed_percent.append(percent_unemployed2)
                number_crimes.append(number_crimes2)
                crime_percent.append(crimes_per_capita2)
                area_counties.append(coord2)
        number_unemployed_column.append(number_unemployed)
        unemployed_percent_column.append(unemployed_percent)
        number_crimes_column.append(number_crimes)
        crime_percent_column.append(crime_percent)
        area_counties_column.append(area_counties)
        count_column.append(len(area_counties))
        highest_unemployment_in_area_column.append(percent_unemployed1 == max(unemployed_percent))
        lowest_unemployment_in_area_column.append(percent_unemployed1 == min(unemployed_percent))
        highest_crime_rate_in_area_column.append(crimes_per_capita1 == max(crime_percent))
        lowest_crime_rate_in_area_column.append(crimes_per_capita1 == min(crime_percent))
    geo_year[f"Number Unemployed for Area Counties ({dist_away} Miles)"] = number_unemployed_column
    geo_year[f"Unemployed Percent for Area Counties ({dist_away} Miles)"] = unemployed_percent_column
    geo_year[f"Number of Crimes for Area Counties ({dist_away} Miles)"] = number_crimes_column
    geo_year[f"Crime Percent for Area Counties ({dist_away} Miles)"] = crime_percent_column
    geo_year[f"Coordinates for Area Counties ({dist_away} Miles)"] = area_counties_column
    geo_year[f"Total Area Counties ({dist_away} Miles)"] = count_column
    geo_year[f"Highest Unemployment in Area ({dist_away} Miles)"] = highest_unemployment_in_area_column
    geo_year[f"Lowest Unemployment in Area ({dist_away} Miles)"] = lowest_unemployment_in_area_column
    geo_year[f"Highest Crime Rate in Area ({dist_away} Miles)"] = highest_crime_rate_in_area_column
    geo_year[f"Lowest Crime Rate in Area ({dist_away} Miles)"] = lowest_crime_rate_in_area_column

    return geo_year

In [17]:
geo_2010 = geo_df[geo_df["Year"]==2010]
dist_away = 50
geo_2010_MO = geo_2010[geo_2010["State"]=="MO"]

mo_test = area_data(geo_2010_MO,100)
mo_test.head()

C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,Year,State,County,Number Unemployed,Percent Unemployed,Number of Crimes,Total Labor Force,Coordinates,Crimes per Capita,Number Unemployed for Area Counties (100 Miles),Unemployed Percent for Area Counties (100 Miles),Number of Crimes for Area Counties (100 Miles),Crime Percent for Area Counties (100 Miles),Coordinates for Area Counties (100 Miles),Total Area Counties (100 Miles),Highest Unemployment in Area (100 Miles),Lowest Unemployment in Area (100 Miles),Highest Crime Rate in Area (100 Miles),Lowest Crime Rate in Area (100 Miles)
7686,2010,MO,Ray,1188.75000,10.28333,839,11571.66667,"(39.3354791, -93.9878427)",0.07250,"[1188.75, 5138.33333, 10468.58333, 213.5, 683....","[10.28333, 9.64167, 8.36667, 7.94167, 12.525, ...","[839, 3880, 4601, 242, 425, 1684, 880, 11203, ...","[0.0725, 0.07272, 0.03681, 0.08988, 0.07799, 0...","[(39.3354791, -93.9878427), (38.6574448, -94.3...",39,False,False,False,False
7692,2010,MO,Cass,5138.33333,9.64167,3880,53355.41667,"(38.6574448, -94.315422)",0.07272,"[1188.75, 5138.33333, 10468.58333, 330.58333, ...","[10.28333, 9.64167, 8.36667, 9.71667, 10.28333...","[839, 3880, 4601, 137, 1712, 880, 721, 1735, 3...","[0.0725, 0.07272, 0.03681, 0.04028, 0.11885, 0...","[(39.3354791, -93.9878427), (38.6574448, -94.3...",33,False,False,False,False
7698,2010,MO,Clay,10468.58333,8.36667,4601,125000.50000,"(39.3176186, -94.479976)",0.03681,"[1188.75, 5138.33333, 10468.58333, 213.5, 683....","[10.28333, 9.64167, 8.36667, 7.94167, 12.525, ...","[839, 3880, 4601, 242, 425, 880, 1735, 75, 348...","[0.0725, 0.07272, 0.03681, 0.08988, 0.07799, 0...","[(39.3354791, -93.9878427), (38.6574448, -94.3...",33,False,False,False,False
7704,2010,MO,Cole,2790.75000,6.85000,909,40684.33333,"(38.5052486, -92.2630393)",0.02234,"[2790.75, 674.33333, 808.25, 1480.66667, 5970....","[6.85, 10.34167, 9.775, 10.28333, 6.44167, 10....","[909, 698, 562, 1712, 11203, 1735, 590, 288, 3...","[0.02234, 0.10719, 0.0682, 0.11885, 0.12104, 0...","[(38.5052486, -92.2630393), (37.544666, -91.44...",45,False,False,False,True
7710,2010,MO,Dade,330.58333,9.71667,137,3401.50000,"(37.48636090000001, -93.9878427)",0.04028,"[5138.33333, 330.58333, 1480.66667, 1507.25, 9...","[9.64167, 9.71667, 10.28333, 9.65833, 11.0, 9....","[3880, 137, 1712, 1649, 880, 721, 1735, 1440, ...","[0.07272, 0.04028, 0.11885, 0.1056, 0.10481, 0...","[(38.6574448, -94.315422), (37.48636090000001,...",28,False,False,False,False


In [22]:
#mo_test.columns
#mo_test["Highest Unemployment in Area (100 Miles)"].value_counts()
mo_test["Total Area Counties (100 Miles)"].min()

14

In [53]:
#mo_test.columns
mo_test['Total Area Counties (50 Miles)'].mean()

10.357798165137615

In [23]:
area_data_2010 = area_data(geo_2010,50)
area_data_2010.head()

C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,Year,State,County,Number Unemployed,Percent Unemployed,Number of Crimes,Total Labor Force,Coordinates,Crimes per Capita,Number Unemployed for Area Counties (50 Miles),Unemployed Percent for Area Counties (50 Miles),Number of Crimes for Area Counties (50 Miles),Crime Percent for Area Counties (50 Miles),Coordinates for Area Counties (50 Miles),Total Area Counties (50 Miles),Highest Unemployment in Area (50 Miles),Lowest Unemployment in Area (50 Miles),Highest Crime Rate in Area (50 Miles),Lowest Crime Rate in Area (50 Miles)
0,2010,AL,Lee,6005.75000,9.02500,7107,66764.16667,"(32.5888702, -85.39627689999999)",0.10645,"[6005.75, 1319.08333, 3320.66667, 584.66667, 2...","[9.025, 14.76667, 9.05833, 11.80833, 10.89167,...","[7107, 1909, 132, 269, 1078, 2894, 901, 2456, ...","[0.10645, 0.2136, 0.0036, 0.05445, 0.04527, 0....","[(32.5888702, -85.39627689999999), (32.3731182...",12,False,False,False,False
6,2010,AL,Bibb,1020.41667,11.42500,658,8934.00000,"(32.9562798, -87.14228949999999)",0.07365,"[1020.41667, 971.66667, 654.5, 3002.41667, 570...","[11.425, 15.09167, 16.84167, 16.875, 17.24167,...","[658, 362, 1, 3401, 612, 5810, 700, 2401, 3231...","[0.07365, 0.05617, 0.00026, 0.19097, 0.18513, ...","[(32.9562798, -87.14228949999999), (32.7859102...",10,False,False,False,False
12,2010,AL,Clay,845.91667,14.46667,208,5851.00000,"(33.279527, -85.84862360000001)",0.03555,"[845.91667, 717.08333, 7261.58333, 5859.5, 219...","[14.46667, 15.40833, 7.10833, 11.36667, 14.925...","[208, 2, 5810, 8222, 2894, 616, 901, 6343, 245...","[0.03555, 0.00043, 0.05675, 0.15947, 0.19615, ...","[(33.279527, -85.84862360000001), (32.8897772,...",12,False,False,False,False
18,2010,AL,Dale,1947.75000,8.83333,2160,22071.25000,"(31.4645298, -85.684578)",0.09786,"[1947.75, 1478.91667, 790.41667, 1816.75, 1196...","[8.83333, 9.44167, 10.91667, 8.60833, 10.375, ...","[2160, 2865, 471, 753, 1047, 963, 269, 1679, 3...","[0.09786, 0.18275, 0.06488, 0.03571, 0.09058, ...","[(31.4645298, -85.684578), (31.8395163, -86.01...",14,False,False,False,False
24,2010,AL,Hale,971.66667,15.09167,362,6445.08333,"(32.7859102, -87.6186379)",0.05617,"[1020.41667, 971.66667, 654.5, 3002.41667, 570...","[11.425, 15.09167, 16.84167, 16.875, 17.24167,...","[658, 362, 1, 3401, 612, 492, 1379, 451, 17670]","[0.07365, 0.05617, 0.00026, 0.19097, 0.18513, ...","[(32.9562798, -87.14228949999999), (32.7859102...",9,False,False,False,False


In [24]:
area_data_2010.to_csv("2010_50_Mile_Area_Data.csv")

In [25]:
area_data_2010.columns

Index(['Year', 'State', 'County', 'Number Unemployed', 'Percent Unemployed',
       'Number of Crimes', 'Total Labor Force', 'Coordinates',
       'Crimes per Capita', 'Number Unemployed for Area Counties (50 Miles)',
       'Unemployed Percent for Area Counties (50 Miles)',
       'Number of Crimes for Area Counties (50 Miles)',
       'Crime Percent for Area Counties (50 Miles)',
       'Coordinates for Area Counties (50 Miles)',
       'Total Area Counties (50 Miles)',
       'Highest Unemployment in Area (50 Miles)',
       'Lowest Unemployment in Area (50 Miles)',
       'Highest Crime Rate in Area (50 Miles)',
       'Lowest Crime Rate in Area (50 Miles)'],
      dtype='object')

In [31]:
area_data_2010["Highest Unemployment in Area (50 Miles)"].value_counts(),area_data_2010["Lowest Unemployment in Area (50 Miles)"].value_counts(),area_data_2010["Highest Crime Rate in Area (50 Miles)"].value_counts(), area_data_2010["Lowest Crime Rate in Area (50 Miles)"].value_counts()

(False    2426
 True      363
 Name: Highest Unemployment in Area (50 Miles), dtype: int64, False    2432
 True      357
 Name: Lowest Unemployment in Area (50 Miles), dtype: int64, False    2391
 True      398
 Name: Highest Crime Rate in Area (50 Miles), dtype: int64, False    2415
 True      374
 Name: Lowest Crime Rate in Area (50 Miles), dtype: int64)

In [32]:
geo_2015 = geo_df[geo_df["Year"]==2015]
area_data_2015 = area_data(geo_2015,50)
area_data_2015.to_csv("2015_50_Mile_Area_Data.csv")

C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\azhol\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py